# This code is to go through each pages eperatly, and get all 42 listings in the pages
*Can be used to get all data as well*

In [ ]:
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os

# Base directory to save CSV files
base_dir = "C:\\Users\\manis\\OneDrive\\Desktop\\UofT PhD Courses\\Fall 2024\\CME538H1 F LEC0101_Data_Scn_CIVMIN\\Big Project\\Project - Safe and affodable housing\\Data\\Housing data\\Web_scrape_Toronto\\from_full_loop"

# Initialize the WebDriver
driver = webdriver.Chrome()  # Ensure you have ChromeDriver in your PATH or specify the path

# Use the provided URL for Toronto real estate listings

url = "https://www.zillow.com/toronto-on/19_p/?searchQueryState=%7B%22pagination%22%3A%7B%22currentPage%22%3A19%7D%2C%22isMapVisible%22%3Atrue%2C%22mapBounds%22%3A%7B%22west%22%3A-79.6177403959961%2C%22east%22%3A-79.1350286040039%2C%22south%22%3A43.515190463936726%2C%22north%22%3A43.900362029998966%7D%2C%22mapZoom%22%3A11%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A792680%2C%22regionType%22%3A6%7D%5D%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%7D%2C%22isListVisible%22%3Atrue%7D"
driver.get(url)
print("Waiting 50 seconds for the page to load fully...")
time.sleep(50)

# Loop through a maximum of 20 pages
for page in range(1, 21):
    # Wait for the page to load and the listings to be present
    WebDriverWait(driver, 300).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'article[data-test="property-card"]'))
    )

    # Prepare a list to hold all listings data for the current page
    listings_data = []

    # Find listings using the correct CSS selector
    listings = driver.find_elements(By.CSS_SELECTOR, 'article[data-test="property-card"]')

    # Loop through listings to extract data
    for listing in listings:
        try:
            address = listing.find_element(By.CSS_SELECTOR, 'address[data-test="property-card-addr"]').text
            price = listing.find_element(By.CSS_SELECTOR, 'span[data-test="property-card-price"]').text
            details = listing.find_elements(By.CSS_SELECTOR, 'ul.StyledPropertyCardHomeDetailsList-c11n-8-106-0__sc-1j0som5-0.eDOePJ li')

            bedrooms = details[0].text if len(details) > 0 else 'N/A'
            bathrooms = details[1].text if len(details) > 1 else 'N/A'
            sqft = details[2].text if len(details) > 2 else 'N/A'

            full_details = ''.join([detail.text for detail in details])
            home_type = full_details.split('-')[-1].strip() if '-' in full_details else 'N/A'

            # Append the extracted data to the listings_data list
            listings_data.append([address, price, bedrooms, bathrooms, sqft, home_type])
        except Exception as e:
            print(f"Error extracting data from a listing: {e}")

    # Define the output file for the current page
    output_file = os.path.join(base_dir, f"zillow_listings_Toronto_Page_{page}.csv")

    # Write the data for the current page to a CSV file
    with open(output_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        # Write the header
        writer.writerow(['Address', 'Price', 'Bedrooms', 'Bathrooms', 'Square Footage', 'Home Type'])
        # Write the data rows
        writer.writerows(listings_data)

    print(f"Data for page {page} saved to {output_file}")

    # Attempt to find the "Next" button and click it to go to the next page
    try:
        next_button = driver.find_element(By.CSS_SELECTOR, 'a[rel="next"][title="Next page"]')
        if next_button.is_enabled() and next_button.is_displayed():
            next_button.click()
            time.sleep(150)  # Adjusted to 2.5 minutes for the next page to load
        else:
            print("No more pages to navigate.")
            break
    except Exception as e:
        print(f"Error finding or clicking the Next button: {e}")
        break

# Close the driver
driver.quit()
